In [1]:
import torch
import torchvision.transforms.v2 as transforms
from torchvision import models
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
import pickle
import numpy as np
import firebase_admin
from firebase_admin import credentials, firestore,storage
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import tempfile
import os

In [2]:
if not firebase_admin._apps:
    cred = credentials.Certificate("imagequest-aab50-firebase-adminsdk-fbsvc-44dd473055.json")
    firebase_admin.initialize_app(cred, {"storageBucket": "imagequest-aab50.firebasestorage.app"})
db = firestore.client()
bucket = storage.bucket()

In [22]:
model =  models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
train_nodes, eval_nodes = get_graph_node_names(model)
print(train_nodes)
print(eval_nodes)

['x', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.relu_1', 'layer1.0.conv3', 'layer1.0.bn3', 'layer1.0.downsample.0', 'layer1.0.downsample.1', 'layer1.0.add', 'layer1.0.relu_2', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.relu_1', 'layer1.1.conv3', 'layer1.1.bn3', 'layer1.1.add', 'layer1.1.relu_2', 'layer1.2.conv1', 'layer1.2.bn1', 'layer1.2.relu', 'layer1.2.conv2', 'layer1.2.bn2', 'layer1.2.relu_1', 'layer1.2.conv3', 'layer1.2.bn3', 'layer1.2.add', 'layer1.2.relu_2', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.relu_1', 'layer2.0.conv3', 'layer2.0.bn3', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.add', 'layer2.0.relu_2', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.relu_1', 'layer2.1.conv3', 'layer2.1.bn3', 'layer2.1.add', 'layer2.1.rel

In [19]:
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToImage(), 
        transforms.ToDtype(torch.float32, scale=True),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

In [25]:
return_nodes = {
    'layer4': 'layer4',
}
features =  create_feature_extractor(model, return_nodes=return_nodes)
collection = db.collection('main2').stream(timeout=300)

In [26]:
extract = []
for x in collection:
    image = x.to_dict()
    image_url = image['url']
    image_id = image['image_id']
    try:
        get_image = requests.get(image_url, timeout=300)
        if get_image.status_code != 200:
            print("CANNOT GET IMAGE")
            continue

        read_image = Image.open(BytesIO(get_image.content)).convert('RGB')
        image_transform = data_transforms(read_image).unsqueeze(0)

        with torch.no_grad():
            output = features(image_transform)

        extraction = output['layer4'].cpu().numpy()
        reduce = extraction.reshape(1, -1)
        extract.append(reduce)

        print(f"FEATURES EXTRACTED: {image_id}: {reduce.shape}")
    except UnidentifiedImageError:
        print("UNIDENTIFIED IMAGE ERROR")
    except Exception as e:
        print("ERROR")
stack_list = np.vstack(extract)

FEATURES EXTRACTED: chess_dataset2/00000021 (2).jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000030 (2).jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000086 (2).jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000007 (5).jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000072 (3).jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000002.JPG: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000052.jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000075 (2).jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000169.jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000112.jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000027.png: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000071 (4).jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000073.jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000028 (2).jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000045.jpg: (1, 100352)
FEATURES EXTRACTED: chess_dataset2/00000194 (2).jpg: 

KeyboardInterrupt: 

In [23]:
with tempfile.NamedTemporaryFile(delete=False, suffix='.pkl') as temp_file:
        pickle.dump(stack_list, temp_file)
        temp_filename = temp_file.name 
blob = bucket.blob("feature_vectors/resnet_features.pkl")
blob.upload_from_filename(temp_filename)
os.remove(temp_filename)